<a href="https://colab.research.google.com/github/iannwtf19/iannwtf19/blob/week11-transformers/Week11/Homework11_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
import datetime

# Data preparation
## Load file
First let's mount Google Drive, where we keep our input file (the bible).
If you are working locally & don't need Google Drive you may skip this cell.
If the file does not exist yet, we will create it.

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.chdir("drive/MyDrive/tensorflow")

Mounted at /content/drive


Next, we download the file if it doesn’t already exist. Finally, open and read it.

In [4]:
import urllib.request
filename = 'bible.txt'
if not os.path.isfile(filename):
    urllib.request.urlretrieve('https://raw.githubusercontent.com/iannwtf19/iannwtf19/main/Week10/bible.txt', 'bible.txt')

In [50]:
# Open input file
text_file = open("bible.txt")
# Read the text file as a string (a sequence of characters)
corpus = text_file.read()

Let's see what the text looks like

In [51]:
print(corpus[:200])

The First Book of Moses:  Called Genesis


1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the


# Tokenization

## Cleaning input text

In [52]:
import re

# Preprocess the text data

preprocessed_text_data = re.sub(r'[^a-zA-Z ]', '', corpus).lower()

# Print an example of the preprocessed text
print(preprocessed_text_data[:500])

the first book of moses  called genesis in the beginning god created the heaven and the earth and the earth was without form and void and darkness was uponthe face of the deep and the spirit of god moved upon the face of thewaters and god said let there be light and there was light and god saw the light that it was good and god divided the lightfrom the darkness and god called the light day and the darkness he called nightand the evening and the morning were the first day and god said let there 


## SentencePieceTokenizer

In [8]:
!pip install "tensorflow-text==2.11.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 45.7 MB/s eta 0:00:00


In [10]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.6 MB/s eta 0:00:00


In [92]:
# Define hyperparameters
vocab_size = 100
m = 32
key_dim = 128
batch_size = 1024

In [93]:
import sentencepiece as sp
import tensorflow_text as tf_text

sp.SentencePieceTrainer.train(input = 'bible.txt', model_prefix = 'sp_tokenizer_model', vocab_size = vocab_size)

# deserialize the trained model file to load it in the correct format
trained_tokenizer_model = tf.io.gfile.GFile('sp_tokenizer_model.model', "rb").read()

# load the model as a tokenizer that can be used inside a tensorflow model
tokenizer = tf_text.SentencepieceTokenizer(
    model=trained_tokenizer_model, out_type=tf.int32, nbest_size=-1, alpha=1, reverse=False,
    add_bos=False, add_eos=False, return_nbest=False, name=None
)

tokenized = tokenizer.tokenize(corpus)

In [94]:
# Create sliding windows of tokenized text with sequence length m+1
windowed_text = tf_text.sliding_window(tokenized, m+1)

In [95]:
train_ds = tf.data.Dataset.from_tensor_slices((windowed_text, windowed_text))
# Shift to the right
train_ds = train_ds.map(lambda x, y: (x[:-1], y[1:]))
train_ds = train_ds.shuffle(vocab_size).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [96]:
print(train_ds)

<PrefetchDataset element_spec=(TensorSpec(shape=(1024, 32), dtype=tf.int32, name=None), TensorSpec(shape=(1024, 32), dtype=tf.int32, name=None))>


# Transformer Model

Let's define our model here. First, the embedding layer
## Embedding Layer

In [97]:
class EmbeddingLayer(tf.keras.layers.Layer):

    def __init__(self, vocab_size, embedding_size, sequence_length):
        super().__init__()
        self.index_embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_size, input_length=1)
        self.positional_embedding_layer = tf.keras.layers.Embedding(sequence_length, embedding_size, input_length=1)

    def call(self, inputs, training=None, mask=None):
        word_index_embeddings = self.index_embedding_layer(inputs)
        sequence_length = tf.shape(inputs)[1]
        positional_indices = tf.range(sequence_length)
        positional_embeddings = self.positional_embedding_layer(positional_indices)
        return word_index_embeddings + positional_embeddings

## Transformer Block

In [98]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_size):
        super().__init__()

        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=key_dim)
        self.dense1 = tf.keras.layers.Dense(units=embedding_size, activation='relu')
        self.dropout1 = tf.keras.layers.Dropout(0.1)
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dense2 = tf.keras.layers.Dense(units=embedding_size)
        self.dropout2 = tf.keras.layers.Dropout(0.1)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=None, mask=None):
        y = self.mha(query=inputs, value=inputs, use_causal_mask=True)
        y = self.dropout1(y, training=training)
        y = inputs + y
        ln_out = self.layer_norm1(y, training=training)
        y = self.dense1(y)
        y = self.dense2(y)
        y = self.dropout2(y, training=training)
        y = ln_out + y
        y = self.layer_norm2(y, training=training)
        return y

## Transformer Model

In [99]:
class TransformerModel(tf.keras.Model):

    def __init__(self, vocab_size, embedding_size, sequence_length, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.sequence_length = sequence_length

        self.optimizer = tf.keras.optimizers.Adam(0.001)
        self.loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        self.metrics_list = [tf.keras.metrics.Mean(name="loss")]

        self.embedding_layer = EmbeddingLayer(vocab_size, embedding_size, sequence_length)
        self.transformer_block = TransformerBlock(embedding_size)
        self.output_layer = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, training=None, mask=None):
        y = self.embedding_layer(inputs)
        y = self.transformer_block(y, training=training)
        y = self.output_layer(y)

        return y

    @tf.function
    def train_step(self, data):

        x, targets = data

        with tf.GradientTape() as tape:
            predictions = self(x, training=True)

            loss = self.loss_function(targets, predictions) + tf.reduce_sum(self.losses)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # update loss metric
        self.metrics[0].update_state(loss)

        # for all metrics except loss, update states (accuracy etc.)
        for metric in self.metrics[1:]:
            metric.update_state(targets, predictions)

        # Return a dictionary mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def generate_text(self, prompt, output_length, top_k):
        prompt = self.tokenizer.tokenize(prompt)

        while len(prompt) <= output_length:
            tokens = tf.expand_dims(prompt, axis = 0)
            tokens = tf.pad(tokens, 
                            tf.constant([[0, 0, ], [self.sequence_length - len(prompt), 0]]))
            
            logit = self.call(tokens)

            logits, indices = tf.math.top_k(logit, k=top_k, sorted=True)
            indices = tf.squeeze(indices)
            logits = tf.squeeze(logits)
            
            next_sample = tf.random.categorical(logits, 1)
            next_sample = tf.squeeze(next_sample)[-1].numpy()

            new_token = indices[-1, next_sample]
            new_token = tf.expand_dims(new_token, axis = -1)
            prompt = tf.concat([prompt, new_token], axis = -1)

        out = self.tokenizer.detokenize(prompt)
        print(out)
        
        return out

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()

Prepare summary writers before training

In [100]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

config_name = f'RUN'
train_log_path = f"logs/{config_name}/{current_time}/train"

train_summary_writer = tf.summary.create_file_writer(train_log_path)

In [101]:
embedding_size = 64
model = TransformerModel(vocab_size, embedding_size, m, tokenizer)

In [102]:
def training_loop(num_epochs):
  for epoch in range(num_epochs): 
      print(f'Epoch {epoch}: ')

      for inputs in tqdm(train_ds):
          metrics = model.train_step(inputs)
    
      gen_text = model.generate_text("Let there be", output_length = 24, top_k = 100)

      with train_summary_writer.as_default(): 
          for metric in model.metrics_list:
              tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
              tf.summary.text(name = "generated text", data = gen_text, step = epoch)

      print([f"train_{key}: {value.numpy()}" for (key, value) in metrics.items()])
      model.reset_metrics()
      print("\n")

num_epochs = 100
training_loop(num_epochs)

Epoch 0: 


100%|██████████| 3031/3031 [02:01<00:00, 24.90it/s]


tf.Tensor(b'Let there begint seeiven thou hann', shape=(), dtype=string)
['train_loss: 2.476088047027588']


Epoch 1: 


100%|██████████| 3031/3031 [01:59<00:00, 25.36it/s]


tf.Tensor(b'Let there behints foret, vel any ho', shape=(), dtype=string)
['train_loss: 2.19230318069458']


Epoch 2: 


...

tf.Tensor(b'Let there be cuty, Lord, hithin the', shape=(), dtype=string)
['train_loss: 1.8950804471969604']


Epoch 98: 


100%|██████████| 3031/3031 [02:02<00:00, 24.67it/s]


tf.Tensor(b'Let there beast, were God. 3:28 And', shape=(), dtype=string)
['train_loss: 1.895329475402832']


Epoch 99: 


100%|██████████| 3031/3031 [02:02<00:00, 24.67it/s]


tf.Tensor(b'Let there beliehds, because were ', shape=(), dtype=string)
['train_loss: 1.894648790359497']




In [103]:
model.save("/content/drive/MyDrive/transformer_model")